<a href="https://colab.research.google.com/github/nbortolotti/tensorflow-experiences/blob/master/colaboratory/Experience_with_iris_dataset_using_tf_data_%26_tf_keras_%26_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#%tensorflow_version 2.x

*tested version TF 2.1.0

# Experience with iris dataset using tf.keras & tensorflow

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

## Data download and dataset creation witout tf.data

In [0]:
train_ds_url = "http://download.tensorflow.org/data/iris_training.csv"
test_ds_url = "http://download.tensorflow.org/data/iris_test.csv"
ds_columns = ['SepalLength', 'SepalWidth','PetalLength', 'PetalWidth', 'Plants']
species = np.array(['Setosa', 'Versicolor', 'Virginica'], dtype=np.object)

## Load data

In [0]:
categories='Plants'

train_path = tf.keras.utils.get_file(train_ds_url.split('/')[-1], train_ds_url)
test_path = tf.keras.utils.get_file(test_ds_url.split('/')[-1], test_ds_url)
    
train = pd.read_csv(train_path, names=ds_columns, header=0)
train_plantfeatures, train_categories = train, train.pop(categories)

test = pd.read_csv(test_path, names=ds_columns, header=0)
test_plantfeatures, test_categories = test, test.pop(categories)

In [0]:
train_plantfeatures

In [0]:
train_categories.value_counts()

In [0]:
y_categorical = tf.keras.utils.to_categorical(train_categories, num_classes=3)
y_categorical_test = tf.keras.utils.to_categorical(test_categories, num_classes=3)

##  Build  the Dataset
from_tensor_slices

To build the dataset we will use tf.data.Dataset set of elements. 

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((train_plantfeatures.values, y_categorical))
dataset = dataset.batch(32)
dataset = dataset.shuffle(1000)
dataset = dataset.repeat()

In [0]:
dataset_test = tf.data.Dataset.from_tensor_slices((test_plantfeatures.values, y_categorical_test))
dataset_test = dataset_test.batch(32)
dataset_test = dataset_test.shuffle(1000)
dataset_test = dataset_test.repeat()

## Build  the Model

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(16, input_dim=4),
  tf.keras.layers.Dense(3, activation=tf.nn.softmax),
])

model.summary()

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

## Train the Model

In [0]:
model.fit(dataset, steps_per_epoch=32, epochs=100, verbose=1)

## Eval the model

In [0]:
loss, accuracy = model.evaluate(dataset_test, steps=32)

print("loss:%f"% (loss))
print("accuracy: %f"%   (accuracy))

## Use the model

If you need to test another specie, you can modify the **new_specie** array.

In [0]:
new_specie = np.array([7.9,3.8,6.4,2.0])
predition = np.around(model.predict(np.expand_dims(new_specie, axis=0))).astype(np.int)[0]
print("This species should be %s" % species[predition.astype(np.bool)][0])

In [0]:
model.predict(np.expand_dims(new_specie, axis=0))

# Save the model

In [0]:
!mkdir model

In [0]:
tf.keras.models.save_model(
    model,
    "./model/iris_model.h5",
    overwrite=True,
    include_optimizer=True
)

In [0]:
new_model = tf.keras.models.load_model("./model/iris_model.h5")

xarray2 = np.array([7.9,3.8,6.4,2.0])

pred = np.around(new_model.predict(np.expand_dims(xarray2, axis=0))).astype(np.int)[0]

print(pred)

print("That means it's a %s" % species[pred.astype(np.bool)][0])